<a href="https://colab.research.google.com/github/AndrewRomitti/Swida/blob/main/LungDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate transformers[torch] > /dev/null 2>&1

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andrewromitti123","key":"faac1459af0bc94cd1618c689bab8fd0"}'}

In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
#!kaggle datasets download -d hamdallak/the-iqothnccd-lung-cancer-dataset
!kaggle datasets download -d kabil007/lungcancer4types-imagedataset

Dataset URL: https://www.kaggle.com/datasets/kabil007/lungcancer4types-imagedataset
License(s): apache-2.0
 96% 114M/119M [00:00<00:00, 129MB/s] 
100% 119M/119M [00:00<00:00, 136MB/s]


In [8]:
#!unzip /content/the-iqothnccd-lung-cancer-dataset.zip > /dev/null 2>&1
!unzip /content/lungcancer4types-imagedataset.zip > /dev/null 2>&1

In [9]:
import datasets
from transformers import DeiTFeatureExtractor

#feature_extractor = DeiTFeatureExtractor.from_pretrained('facebook/deit-base-distilled-patch16-224')

#ds = datasets.load_dataset("imagefolder",data_dir="/content/The IQ-OTHNCCD lung cancer dataset")
ds = datasets.load_dataset("imagefolder",data_dir="/content/Data")

Resolving data files:   0%|          | 0/613 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/315 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
ds["train"] = ds["train"].shuffle(seed=42)
#ds = ds["train"].train_test_split(test_size=0.2)

In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 613
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 72
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 315
    })
})

In [12]:
import torch
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

train_tfs = Compose(
    [
    Resize(size=(224,224)),
    ToTensor(),
    ]
    )

test_tfs = Compose(
    [
      Resize(size=(224,224)),
      ToTensor(),
    ]
)

def transforms(example):
  example["pixel_values"] = [train_tfs(image.convert("RGB")) for image in example["image"]]
  del example["image"]
  return example

In [13]:
ds = ds.with_transform(transforms)

In [14]:
train_ds = ds["train"]
test_ds = ds["test"]
valid_ds = ds["validation"]

In [15]:
import transformers
checkpoint = "facebook/deit-base-distilled-patch16-224"

image_processor = transformers.AutoImageProcessor.from_pretrained(checkpoint, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

In [16]:
data_collator = transformers.DefaultDataCollator(return_tensors="pt")

In [17]:
import evaluate
import numpy as np

f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return {"accuracy": np.mean(predictions == labels)}

In [18]:
labels = ds["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [19]:
from transformers import DeiTConfig, DeiTModel

model = transformers.DeiTForImageClassification.from_pretrained(checkpoint,
                                                                num_labels=len(labels),
                                                                id2label=id2label,
                                                                label2id=label2id,
                                                                ignore_mismatched_sizes=True,)

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/349M [00:00<?, ?B/s]

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
args = transformers.TrainingArguments (
    output_dir = "lungcancer_model_deit",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=10,
    eval_accumulation_steps=10,
    num_train_epochs=15,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_strategy="epoch",
    seed=1234,
    logging_steps=7,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
import torch

device = torch.device("cuda")
print(device)

cuda


In [22]:
trainer = transformers.Trainer(
    model = model,
    args=args,
    data_collator = data_collator,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

In [23]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
0,1.362200,1.125933,0.458333
1,0.735000,0.946083,0.500000
2,0.528700,0.900607,0.597222
3,0.282200,0.723645,0.666667
4,0.145900,0.532656,0.777778
5,0.068700,0.422565,0.833333
6,0.026900,0.413449,0.833333
7,0.013900,0.383632,0.833333
8,0.007700,0.530650,0.819444
10,0.003800,0.418193,0.847222


TrainOutput(global_step=105, training_loss=0.21303045448093189, metrics={'train_runtime': 426.6045, 'train_samples_per_second': 21.554, 'train_steps_per_second': 0.246, 'total_flos': 6.479511591845376e+17, 'train_loss': 0.21303045448093189, 'epoch': 13.636363636363637})

In [ ]:
#check-100 84% accurate

In [54]:
checkpoint = "facebook/deit-base-distilled-patch16-224"
model_checkpoint = "andrewromitti/Swida"

validation_model = transformers.DeiTForImageClassification.from_pretrained(model_checkpoint,torch_dtype=torch.float16)
image_processor = transformers.AutoImageProcessor.from_pretrained(checkpoint, return_tensors="pt")

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [55]:
from transformers import pipeline

classifier = pipeline("image-classification", model=validation_model, image_processor=image_processor)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [56]:
image = "/content/Data/valid/normal/004162_01_01_150.png"
results = classifier(image)

In [57]:
max_result = max(results, key=lambda x: x['score'])
predicted_label = max_result['label']
print(predicted_label)

normal
